<a href="https://colab.research.google.com/github/marcelonieva7/stock-markets-analytics-zoomcamp/blob/main/HM_week_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [160]:
# install main library YFinance
!pip install yfinance

In [159]:
# IMPORTS
import numpy as np
import pandas as pd

#Fin Data Sources
import yfinance as yf
import pandas_datareader as pdr

import time
from datetime import date


## 1) GDPC

In [161]:
gdpc = pdr.DataReader("GDPC1", "fred", start="2022-01-01")

In [162]:
gdpc['gdpc_us_yoy'] = gdpc.GDPC1/gdpc.GDPC1.shift(4)-1
gdpc.tail()

,GDPC1,gdpc_us_yoy
DATE,,
2022-10-01,21989.981,NaN
2023-01-01,22112.329,0.017179
2023-04-01,22225.350,0.023825
2023-07-01,22490.692,0.029269
2023-10-01,22679.255,0.031345


In [163]:
round(gdpc[-4:].gdpc_us_yoy.mean() * 100, 1)

2.5

## 2) Find the min value of (dgs10-dgs2)

In [164]:
dgs2 = pdr.DataReader("DGS2", "fred", start="2000-01-01")
dgs10 = pdr.DataReader("DGS10", "fred", start="2000-01-01")

In [165]:
dgs10.tail(), dgs2.tail()

(            DGS10
 DATE             
 2024-04-16   4.67
 2024-04-17   4.59
 2024-04-18   4.64
 2024-04-19   4.62
 2024-04-22   4.62,
             DGS2
 DATE            
 2024-04-16  4.97
 2024-04-17  4.93
 2024-04-18  4.98
 2024-04-19  4.97
 2024-04-22  4.97)

In [166]:
dgs_join = dgs2.join(dgs10, lsuffix='_dgs2', rsuffix='_dgs10')

In [167]:
dgs_join["diff"] = dgs_join.DGS10 - dgs_join.DGS2

In [168]:
dgs_join["diff"].min().round(1)

-1.1

## 3) Which Index is better recently?

In [169]:
spx = yf.download(tickers = ["^GSPC", "^MXX"],
                     start = "2019-04-09",
                     end = "2024-04-10",
                     interval = "1d")

[*********************100%%**********************]  2 of 2 completed


In [170]:
spx.loc["2024-04-09"]["Adj Close"] / spx.loc["2019-04-09"]["Close"] - 1

Ticker
^GSPC    0.810128
^MXX     0.275062
dtype: float64

## 4)  52-weeks range ratio (2023) for the selected stocks

In [75]:
top_profit = yf.download(tickers = ["2222.SR" ,"BRK-B" ,"AAPL" ,"MSFT" ,"GOOG" ,"JPM"],
                     start = "2023-01-01",
                     end = "2024-01-01",
                     interval = "1d")

[*********************100%%**********************]  6 of 6 completed


In [76]:
top_profit.tail()

Price       Adj Close                                                  \
Ticker        2222.SR        AAPL       BRK-B        GOOG         JPM   
Date                                                                    
2023-12-26  32.877789  192.803986  356.829987  142.820007  166.387451   
2023-12-27  32.977268  192.903839  356.950012  141.440002  167.385437   
2023-12-28  32.877789  193.333298  357.570007  141.279999  168.274734   
2023-12-29        NaN  192.284637  356.660004  140.929993  168.077118   
2023-12-31  32.828049         NaN         NaN         NaN         NaN   

Price                       Close                                      ...  \
Ticker            MSFT    2222.SR        AAPL       BRK-B        GOOG  ...   
Date                                                                   ...   
2023-12-26  373.968445  33.049999  193.050003  356.829987  142.820007  ...   
2023-12-27  373.379547  33.150002  193.149994  356.950012  141.440002  ...   
2023-12-28  374.587280  33.049999  193.580002  357.570007  141.279999  ...   
2023-12-29  375.345886        NaN  192.529999  356.660004  140.929993  ...   
2023-12-31         NaN  33.000000         NaN         NaN         NaN  ...   

Price             Open                                          Volume  \
Ticker           BRK-B        GOOG         JPM        MSFT     2222.SR   
Date                                                                     
2023-12-26  356.899994  142.979996  167.460007  375.000000  14598967.0   
2023-12-27  355.929993  142.830002  167.839996  373.690002  14815683.0   
2023-12-28  357.480011  141.850006  169.350006  375.369995  12720450.0   
2023-12-29  357.559998  140.679993  170.000000  376.000000         NaN   
2023-12-31         NaN         NaN         NaN         NaN  10776909.0   

Price                                                                 
Ticker            AAPL      BRK-B        GOOG        JPM        MSFT  
Date                                                                  
2023-12-26  28919300.0  1964400.0  11170100.0  4683600.0  12673100.0  
2023-12-27  48087700.0  2478500.0  17288400.0  6428600.0  14905400.0  
2023-12-28  34049900.0  2740600.0  12192500.0  6320100.0  14327000.0  
2023-12-29  42628800.0  3269400.0  14872700.0  6431800.0  18723000.0  
2023-12-31         NaN        NaN         NaN        NaN         NaN  

[5 rows x 36 columns]

In [85]:
((top_profit["Adj Close"].max() - top_profit["Adj Close"].min()) / top_profit["Adj Close"].max()).sort_values().round(2)

Ticker
BRK-B      0.21
2222.SR    0.21
JPM        0.28
AAPL       0.37
GOOG       0.39
MSFT       0.42
dtype: float64

## 5) Dividend Yield

In [146]:
tickers = yf.Tickers(["2222.SR" ,"BRK-B" ,"AAPL" ,"MSFT" ,"GOOG" ,"JPM"])

In [147]:
for name, tick in tickers.tickers.items():
  print(name, " ", tick.dividends[tick.dividends.index.year == 2023].sum())

2222.SR   0.9107640000000001
BRK-B   0.0
AAPL   0.95
MSFT   2.79
GOOG   0.0
JPM   4.05


In [158]:
for name, tick in tickers.tickers.items():
  print("==========================")
  print(name)
  dividends = tick.dividends[tick.dividends.index.year == 2023].sum()
  year_close = tick.history(start="2023-01-01", end="2024-01-01").Close.values[-1]
  print("dividends ", dividends)
  print("year close ", year_close)
  print("yield ", round(dividends / year_close * 100, 1))

2222.SR
dividends  0.9107640000000001
year close  32.82804870605469
yield  2.8
BRK-B
dividends  0.0
year close  356.6600036621094
yield  0.0
AAPL
dividends  0.95
year close  192.28463745117188
yield  0.5
MSFT
dividends  2.79
year close  375.34588623046875
yield  0.7
GOOG
dividends  0.0
year close  140.92999267578125
yield  0.0
JPM
dividends  4.05
year close  168.07711791992188
yield  2.4
